In [ ]:
#%pip install imblearn

In [16]:
import numpy as np
import pandas as pd
import pickle
from lazypredict.Supervised import LazyRegressor 
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
import warnings
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
warnings.filterwarnings('ignore')

In [17]:
df = pd.read_csv(r"C:\Users\manue\OneDrive\Escritorio\Ironhack\Proyectos\Final-Project\Data\Fraud.csv\Fraud.csv")

In [18]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0


In [19]:
le = LabelEncoder()

df['type'] = le.fit_transform(df['type'])

In [20]:
df['nameOrig'] = le.fit_transform(df['nameOrig'])

In [21]:
df['nameDest'] = le.fit_transform(df['nameDest'])

In [22]:
def preprocess_dataframe(df):
    # Contar valores nulos por columna
    null_counts = df.isnull().sum()
    print(f"Valores nulos por columna:\n{null_counts}\n")
    
    # Información del dataframe
    print("Información del dataframe:")
    df.info()
    print()
    
    # Cambiar nombres de columnas a minúsculas y reemplazar espacios por guiones bajos
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]
    
    return df

In [8]:
preprocessed_df = preprocess_dataframe(df)

Valores nulos por columna:
step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

Información del dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            int32  
 2   amount          float64
 3   nameOrig        int32  
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        int32  
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int32(3), int64(3)
memory usage: 461.2 MB



In [9]:
preprocessed_df

,step,type,amount,nameorig,oldbalanceorg,newbalanceorig,namedest,oldbalancedest,newbalancedest,isfraud,isflaggedfraud
0,1,3,9839.64,757869,170136.00,160296.36,1662094,0.00,0.00,0,0
1,1,3,1864.28,2188998,21249.00,19384.72,1733924,0.00,0.00,0,0
2,1,4,181.00,1002156,181.00,0.00,439685,0.00,0.00,1,0
3,1,1,181.00,5828262,181.00,0.00,391696,21182.00,0.00,1,0
4,1,3,11668.14,3445981,41554.00,29885.86,828919,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,1,339682.13,5651847,339682.13,0.00,505863,0.00,339682.13,1,0
6362616,743,4,6311409.28,1737278,6311409.28,0.00,260949,0.00,0.00,1,0
6362617,743,1,6311409.28,533958,6311409.28,0.00,108224,68488.84,6379898.11,1,0
6362618,743,4,850002.52,2252932,850002.52,0.00,319713,0.00,0.00,1,0


In [ ]:
preprocessed_df.to_csv(r"C:\Users\manue\OneDrive\Escritorio\Ironhack\Proyectos\Final-Project\Data wrangling\preprocessed_df.csv", index=False)

## Probé el modelo sin equilibrar los datos y al haber muy pocos casos de fraude no está bien entrenado para detectarlos, produce muchos falsos negativos

In [ ]:
df2 = preprocessed_df

X = preprocessed_df.drop(columns='isfraud')

y = preprocessed_df['isfraud']

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test  = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)

y_pred = log_reg.predict(X_test)

confusion_matrix(y_test,y_pred)

accuracy_score(y_test, y_pred)

In [23]:
X = preprocessed_df.drop(columns='isfraud')

y = preprocessed_df['isfraud']

In [ ]:
y_pred = log_reg.predict(X_test)

print(classification_report(y_test, y_pred))

## Utilizamos SMOTE para introducir samples sinteticos a la muestra

In [24]:
over_sample = SMOTE(random_state=0)
X,y = over_sample.fit_resample(X,y)

In [25]:
y.value_counts()

0    6354407
1    6354407
Name: isfraud, dtype: int64

### Despues de utilizar SMOTE tenemos una prediccion mucho mas real

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=0)

log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)

y_pred = log_reg.predict(X_test)

confusion_matrix(y_test,y_pred)

accuracy_score(y_test, y_pred)

y_pred = log_reg.predict(X_test)

print(classification_report(y_test, y_pred))

## Probaremos el cross validation score para ver como se comporta el modelo con 3 tipos de testeos diferentes

In [ ]:
from sklearn.model_selection import cross_val_score
cvs = cross_val_score(log_reg,X,y,cv=3)
print(cvs)

In [ ]:
cvs.mean()

## Elimino la columna de nameorig por tener demasiados valores unicos y una correlacion de 0

X = preprocessed_df[['step', 'type', 'amount', 'oldbalanceorg', 'newbalanceorig','namedest', 'oldbalancedest', 'isflaggedfraud']]
y = preprocessed_df['isfraud']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)

y_pred = log_reg.predict(X_test)

y_pred

confusion_matrix(y_test,y_pred)

accuracy_score(y_test, y_pred)

0.9485569729151285

#### Aumenta hasta 0.99 la precision del modelo

In [ ]:
with open('model_weights.pkl', 'wb') as file:
    pickle.dump(log_reg.coef_, file)

In [27]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Define the XGBoost model
xgb_model = xgb.XGBClassifier()

# Train the XGBoost model
xgb_model.fit(X_train, y_train)

# Make predictions on the test dataset using the trained XGBoost model
y_pred = xgb_model.predict(X_test)

# Evaluate the accuracy of the model on the test dataset
confusion = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print('Confusion Matrix:\n', confusion)
print('Accuracy Score:', accuracy)

Confusion Matrix:
 [[1902400    3575]
 [   1706 1904964]]
Accuracy Score: 0.9986148723523958
